# Question Answering using Embeddings

Many use cases require GPT-3 to respond to user questions with insightful answers. For example, a customer support chatbot may need to provide answers to common questions. The GPT models have picked up a lot of general knowledge in training, but we often need to ingest and use a large library of more specific information.

In this notebook we will demonstrate a method for enabling GPT-3 able to answer questions using a library of text as a reference, by using document embeddings and retrieval. We'll be using a dataset of Wikipedia articles about the 2020 Summer Olympic Games. Please see [this notebook](fine-tuned_qa/olympics-1-collect-data.ipynb) to follow the data gathering process.

In [76]:
import pandas as pd
import openai
import numpy as np
import pickle
from transformers import GPT2TokenizerFast
import json

COMPLETIONS_MODEL = "text-davinci-002"

In [6]:
openai.api_type = "azure"
openai.api_base = "https://YOURINSTANCENAMEHERE.openai.azure.com/"
openai.api_version = "2022-06-01-preview"
openai.api_key = "YOURKEYHERE"

By default, GPT-3 isn't an expert on the 2020 Olympics:

In [8]:
prompt = "Who won the 2020 Summer Olympics men's high jump?"

openai.Completion.create(
    engine="jota_jnj_poc", # Jota -- added
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"The 2020 Summer Olympics men's high jump was won by Mariusz Przybylski of Poland."

Mariusz Przybylski is a professional footballer from Poland, and not much of a high jumper! Evidently GPT-3 needs some assistance here. 

The first issue to tackle is that the model is hallucinating an answer rather than telling us "I don't know". This is bad because it makes it hard to trust the answer that the model gives us! 

# 0) Preventing hallucination with prompt engineering

We can address this hallucination issue by being more explicit with our prompt:


In [9]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    engine="jota_jnj_poc", # Jota -- added
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Sorry, I don't know."

To help the model answer the question, we provide extra contextual information in the prompt. When the total required context is short, we can include it in the prompt directly. For example we can use this information taken from Wikipedia. We update the initial prompt to tell the model to explicitly make use of the provided text.

In [10]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places 
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics. 
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a 
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and 
Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
of Sweden (1984 to 1992).

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    engine="jota_jnj_poc", # Jota -- added
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Gianmarco Tamberi and Mutaz Essa Barshim won the 2020 Summer Olympics men's high jump."

Adding extra information into the prompt only works when the dataset of extra content that the model may need to know is small enough to fit in a single prompt. What do we do when we need the model to choose relevant contextual information from within a large body of information?

**In the remainder of this notebook, we will demonstrate a method for augmenting GPT-3 with a large body of additional contextual information by using document embeddings and retrieval.** This method answers queries in two steps: first it retrieves the information relevant to the query, then it writes an answer tailored to the question based on the retrieved information. The first step uses the [Embedding API](https://beta.openai.com/docs/guides/embeddings), the second step uses the [Completions API](https://beta.openai.com/docs/guides/completion/introduction).
 
The steps are:
* Preprocess the contextual information by splitting it into chunks and create an embedding vector for each chunk.
* On receiving a query, embed the query in the same vector space as the context chunks and find the context embeddings which are most similar to the query.
* Prepend the most relevant context embeddings to the query prompt.
* Submit the question along with the most relevant context to GPT, and receive an answer which makes use of the provided contextual information.

# 1) Preprocess the document library

We plan to use document embeddings to fetch the most relevant part of parts of our document library and insert them into the prompt that we provide to GPT-3. We therefore need to break up the document library into "sections" of context, which can be searched and retrieved separately. 

Sections should be large enough to contain enough information to answer a question; but small enough to fit one or several into the GPT-3 prompt. We find that approximately a paragraph of text is usually a good length, but you should experiment for your particular use case. In this example, Wikipedia articles are already grouped into semantically related headers, so we will use these to define our sections. This preprocessing has already been done in [this notebook](examples/fine-tuned_qa/olympics-1-collect-data.ipynb), so we will load the results and use them.

In [11]:
# We have hosted the processed dataset, so you can download it directly without having to recreate it.
# This dataset has already been split into sections, one row for each section of the Wikipedia page.

df = pd.read_csv('https://cdn.openai.com/API/examples/data/olympics_sections_text.csv')
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(5)

3964 rows in the data.


,,content,tokens
title,heading,,
China at the 2020 Summer Olympics,Modern pentathlon,Chinese athletes qualified for the following s...,72
Malawi at the 2020 Summer Olympics,Summary,Malawi competed at the 2020 Summer Olympics in...,96
Boxing at the 2020 Summer Olympics – Men's heavyweight,Competition format,"Like all Olympic boxing events, the competitio...",223
Australia at the 2020 Summer Olympics,Sprint,Australian canoeists qualified a total of six ...,159
Sailing at the 2020 Summer Olympics – 49er FX,Summary,The Women 49er FX is a sailing event on the Sa...,303


We preprocess the document sections by creating an embedding vector for each section. An embedding is a vector of numbers that helps us understand how semantically similar or different the texts are. The closer two embeddings are to each other, the more similar are their contents. See the [documentation on OpenAI embeddings](https://beta.api.openai.org/docs/guides/embeddings/) for more information.

This indexing stage can be executed offline and only runs once to precompute the indexes for the dataset so that each piece of content can be retrieved later. Since this is a small example, we will store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like [Pinecone](https://www.pinecone.io/) or [Weaviate](https://github.com/semi-technologies/weaviate) to power the search.

For the purposes of this tutorial we chose to use Curie embeddings, which are 4096-dimensional embeddings at a very good price and performance point. Since we will be using these embeddings for retrieval, we’ll use the "search" embeddings (see the [documentation](https://beta.api.openai.org/docs/guides/embeddings/)).

In [12]:
MODEL_NAME = "curie"

DOC_EMBEDDINGS_MODEL = f"text-search-{MODEL_NAME}-doc-001"
QUERY_EMBEDDINGS_MODEL = f"text-search-{MODEL_NAME}-query-001"


In [21]:
def get_doc_embedding(text: str): # -> list[float]:
    result = openai.Embedding.create(
      engine="jota_jnj_ts_curi_doc", # Jota -- added
      model=DOC_EMBEDDINGS_MODEL,
      input=text
    )
    return result["data"][0]["embedding"]

def get_query_embedding(text: str): # -> list[float]:
    result = openai.Embedding.create(
      engine="jota_jnj_ts_curi_doc", # Jota -- added
      model=QUERY_EMBEDDINGS_MODEL,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame): # -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_doc_embedding(r.content.replace("\n", " ")) for idx, r in df.iterrows()
    }

In [22]:
def load_embeddings(fname: str): # -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

Again, we have hosted the embeddings for you so you don't have to re-calculate them from scratch.

In [23]:
#document_embeddings = load_embeddings("https://cdn.openai.com/API/examples/data/olympics_sections_document_embeddings.csv")

# ===== OR, uncomment the below line to recaculate the embeddings from scratch. ========

context_embeddings = compute_doc_embeddings(df) # jota - takes 5m30s
document_embeddings = context_embeddings # jota added to avoid renaming in the code below


In [50]:
# An example embedding:
example_entry = list(document_embeddings.items())[0] #replace with document_embeddings if using the load_embeddings call
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

# dict[tuple[str, str], list[float]]:
print("Vector size:", len(example_entry[1]))

('2020 Summer Olympics', 'Summary') : [-0.001217024284414947, 0.003676572348922491, 0.0007319531869143248, 0.0093699274584651, -0.009624923579394817]... (4096 entries)
Vector size: 4096


# Microsoft/Insert the embeddings into CosmosDB

In [99]:
#do imports, recourse on the list and add items to the collection after formatting the right way

import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import datetime
import uuid

HOST = "https://jotajnjembeddings.documents.azure.com:443/"
MASTER_KEY = "INSERT YOUR COSMOSDB KEY HERE"
DATABASE_ID = "vectordb"
CONTAINER_ID = "embeddings"

client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY} )

try:
        # setup database for this sample
        db = client.create_database_if_not_exists(id=DATABASE_ID)
        # setup container for this sample
        container = db.create_container_if_not_exists(id=CONTAINER_ID, partition_key=PartitionKey(path='/tag', kind='Hash'))

        print('Inserting embeddings')

        record_count = 1
        for embedding in list(document_embeddings.items()):
            record = {}
            # /tag is the partition key in CosmosDB
            record["tag"] = embedding[0][1]

            # /id is a mandatory field, fill it with a GUID
            record["id"] = str(uuid.uuid4())

            # applicational id, I'm just prefixing the title of the text with a numeric, as there are duplicate titles
            record["docid"] = str(record_count) + "/" + embedding[0][0]

            # the embedding vector
            record["embeddings"] = embedding[1] 

            print(record["id"], record["docid"])

            container.create_item(body=record)

            record_count += 1
            # print(record)
            # print(json.dumps(example_entry))

        print("Total embeddings inserted was", record_count-1)

except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

Inserting embeddings
fc45729c-e1c3-4afe-b651-880b98d223d0 1/2020 Summer Olympics
66ad457c-08d6-4a5c-bc0d-3b6b315eb4b8 2/2020 Summer Olympics
a13aaa82-df68-4769-9b7a-32a92ace7818 3/2020 Summer Olympics
388dd496-bed2-4151-be8f-d1f277f55ad3 4/2020 Summer Olympics
0408701d-fc41-4daa-bba1-5b95c4e559a3 5/2020 Summer Olympics
bcd4b5b4-cae9-452a-919f-35df5f3c5ea3 6/2020 Summer Olympics
8deb1565-718a-4b7f-a41f-f50b3289c3e9 7/2020 Summer Olympics
c18f7ecc-a360-498b-829f-3bb84c42b9ac 8/2020 Summer Olympics
5a2ec03c-3131-4fa8-8cae-27fda85987d0 9/2020 Summer Olympics
89dae9dd-4d8d-4159-8864-9746060e5942 10/2020 Summer Olympics
72332746-d717-47c9-acd8-eeea43297430 11/2020 Summer Olympics
1a7c570b-9d32-4d6b-8de7-ed3eebe6ebad 12/2020 Summer Olympics
415130d2-433e-48a9-9b41-e43538efcf3a 13/2020 Summer Olympics
9db43282-85c1-4a64-a1ba-813c14ac3293 14/2020 Summer Olympics
f38c05de-b830-4ad6-b5a9-1f971f55fe26 15/2020 Summer Olympics
88e26921-d736-404b-8e6e-24e2f1261b44 16/2020 Summer Olympics
892012c9-7a6

# Microsoft/Do a simple query using pre-obtained embeddings against the ones stored

Note: not using the ones returned by the OpenAI embeddings query, for time, but the code is equivalent

In [135]:
from timeit import default_timer as timer

print('Querying for an Item by cosinesim')

start = timer()

# enable_cross_partition_query should be set to True as the container is partitioned
items = list(container.query_items(
    query="SELECT TOP 5 embeddings.id, embeddings.docid, udf.cosinesim(embeddings.embeddings, @query_embeddings) AS cosim FROM embeddings ORDER BY embeddings.cosim DESC",
    parameters=[
        { "name":"@query_embeddings", "value": example_entry[1] }
        #, { "name":"@tag", "value": "Summary" } -- if filter by tag desired, see also comment below
    ],
    enable_cross_partition_query=True
))

# note: to add a query by tag, simply add "WHERE embeddings.tag = @tag" to the SQL query, where tag is a parameter passed in the same way as @query_embeddings is

end = timer()

print("Elapsed time in seconds:", end-start)
print(items)

Querying for an Item by cosinesim
Elapsed time in seconds: 0.20460292299685534
[{'id': '6357995b-68f3-478c-be4c-8a3971cce665', 'docid': '3964/Haiti at the 2020 Summer Olympics', 'cosim': 0.6268227870795141}, {'id': '0390bb3b-211b-432c-a69c-0121ab2bd5a8', 'docid': '3963/Haiti at the 2020 Summer Olympics', 'cosim': 0.6706760445812}, {'id': 'ceff94f1-d0b5-42e9-b18d-cea274faacf6', 'docid': '3962/Haiti at the 2020 Summer Olympics', 'cosim': 0.61331972116933}, {'id': 'cdbf22f4-ac60-4154-a85d-cc3d43b0eec2', 'docid': '3961/Haiti at the 2020 Summer Olympics', 'cosim': 0.6125618888869582}, {'id': '6f125f6e-f763-465e-bcde-148c75cea73b', 'docid': '3960/Haiti at the 2020 Summer Olympics', 'cosim': 0.7940232539807472}]


# THE CODE BELOW IS ADAPTED FROM THE OPENAI NOTEBOK TO RUN WITH AZURE OPENAI, BUT IT'S NOT MODIFIED TO USE COSMOSDB -- the above ilustrates the approach

So we have split our document library into sections, and encoded them by creating embedding vectors that represent each chunk. Next we will use these embeddings to answer our users' questions.

# 2) Find the most similar document embeddings to the question embedding

At the time of question-answering, to answer the user's query we compute the query embedding of the question and use it to find the most similar document sections. Since this is a small example, we store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like [Pinecone](https://www.pinecone.io/) or [Weaviate](https://github.com/semi-technologies/weaviate) to power the search.

In [47]:
#def vector_similarity(x: list[float], y: list[float]): # -> float:
def vector_similarity(x, y): # -> float:
    """
    We could use cosine similarity or dot product to calculate the similarity between vectors.
    In practice, we have found it makes little difference. 
    """
    return np.dot(np.array(x), np.array(y))

#def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]): # -> list[(float, (str, str))]:
def order_document_sections_by_query_similarity(query, contexts): # -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_query_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [51]:
order_document_sections_by_query_similarity("Who won the men's high jump?", document_embeddings)[:5]

[(0.6054076060277246,
  ("Athletics at the 2020 Summer Olympics – Women's 1500 metres", 'Records')),
 (0.5985058629658035,
  ("Athletics at the 2020 Summer Olympics – Women's triple jump", 'Records')),
 (0.5947608691846494,
  ('United States at the 2020 Summer Olympics', 'Medalists')),
 (0.5938955276930331,
  ("Athletics at the 2020 Summer Olympics – Women's 100 metres hurdles",
   'Records')),
 (0.5927976130078663,
  ("Athletics at the 2020 Summer Olympics – Men's 400 metres hurdles",
   'New records'))]

In [52]:
order_document_sections_by_query_similarity("Who won the women's high jump?", document_embeddings)[:5]

[(0.6030094304897664,
  ("Athletics at the 2020 Summer Olympics – Women's 1500 metres", 'Records')),
 (0.5976327360324093,
  ("Athletics at the 2020 Summer Olympics – Women's high jump", 'Summary')),
 (0.5969583388353468,
  ('United States at the 2020 Summer Olympics', 'Medalists')),
 (0.5967197412016225,
  ("Athletics at the 2020 Summer Olympics – Women's triple jump", 'Summary')),
 (0.5940997166087074,
  ("Athletics at the 2020 Summer Olympics – Women's triple jump", 'Records'))]

We can see that the most relevant document sections for each question are the summaries for the Men's and Women's high jump competitions - which is exactly what we would expect.

# 3) Add the most relevant document sections to the query prompt

Once we've calculated the most relevant pieces of context, we construct a prompt by simply prepending them to the supplied query. It is helpful to use a query separator to help the model distinguish between separate pieces of text.

In [53]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
separator_len = len(tokenizer.tokenize(SEPARATOR))

f"Context separator contains {separator_len} tokens"

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

'Context separator contains 3 tokens'

In [54]:
#def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
def construct_prompt(question, context_embeddings, df) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [55]:
prompt = construct_prompt(
    "Who won the 2020 Summer Olympics men's high jump?",
    document_embeddings,
    df
)

print("===\n", prompt)

Selected 5 document sections:
("Athletics at the 2020 Summer Olympics – Women's high jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's long jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Women's triple jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's long jump", 'Background')
("Athletics at the 2020 Summer Olympics – Men's triple jump", 'Summary')
===
 Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

* The women's high jump event at the 2020 Summer Olympics took place on 5 and 7 August 2021 at the Japan National Stadium. Even though 32 athletes qualified through the qualification system for the Games, only 31 took part in the competition. This was the 22nd appearance of the event, having appeared at every Olympics since women's athletics was introduced in 1928.
* The men's long jump event at the 2020 Summer Olympics took place b

We have now obtained the document sections that are most relevant to the question. As a final step, let's put it all together to get an answer to the question.

# 4) Answer the user's question based on the context.

Now that we've retrieved the relevant context and constructed our prompt, we can finally use the Completions API to answer the user's query.

In [56]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [59]:
#def answer_query_with_context(
#    query: str,
#    df: pd.DataFrame,
#    document_embeddings: dict[(str, str), np.array],
#    show_prompt: bool = False
#) -> str:
def answer_query_with_context(
    query,
    df,
    document_embeddings,
    show_prompt = False
):
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                engine="jota_jnj_poc",
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [60]:
answer_query_with_context("Who won the 2020 Summer Olympics men's high jump?", df, document_embeddings)

Selected 5 document sections:
("Athletics at the 2020 Summer Olympics – Women's high jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's long jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Women's triple jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's long jump", 'Background')
("Athletics at the 2020 Summer Olympics – Men's triple jump", 'Summary')


"I don't know."

Wow! By combining the Embeddings and Completions APIs, we have created a question-answering model which can answer questions using a large base of additional knowledge. It also understands when it doesn't know the answer! 

For this example we have used a dataset of Wikipedia articles, but that dataset could be replaced with books, articles, documentation, service manuals, or much much more. **We can't wait to see what you create with GPT-3!**

# More Examples

Let's have some fun and try some more examples.

In [61]:
query = "Why was the 2020 Summer Olympics originally postponed?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 8 document sections:
('List of 2020 Summer Olympics medal winners', 'Summary')
('South Korea at the 2020 Summer Olympics', 'Summary')
('Surfing at the 2020 Summer Olympics', 'Summary')
('Venues of the 2020 Summer Olympics and Paralympics', 'Summary')
('Bids for the 2020 Summer Olympics', 'Summary')
('Oman at the 2020 Summer Olympics', 'Summary')
('Swimming at the 2020 Summer Olympics – Qualification', 'Summary')
('Yemen at the 2020 Summer Olympics', 'Summary')

Q: Why was the 2020 Summer Olympics originally postponed?
A: The 2020 Summer Olympics were originally postponed because of the COVID-19 pandemic.


In [62]:
query = "In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 8 document sections:
('United States at the 2020 Summer Olympics', 'Medalists')
('List of 2020 Summer Olympics medal winners', 'Summary')
('List of divers at the 2020 Summer Olympics', 'Summary')
('Malaysia at the 2020 Summer Olympics', 'Competitors')
("2020 FIBA Men's Olympic Qualifying Tournaments", 'Format')
('Brazil at the 2020 Summer Olympics', 'Competitors')
('Hong Kong at the 2020 Summer Olympics', 'Summary')
('2020 Summer Olympics marketing', 'Corporate sponsorship and advertising')

Q: In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?
A: I don't know.


In [63]:
query = "What was unusual about the men’s shotput competition?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 8 document sections:
("Athletics at the 2020 Summer Olympics – Men's 400 metres hurdles", 'New records')
("Athletics at the 2020 Summer Olympics – Women's 1500 metres", 'Records')
("Athletics at the 2020 Summer Olympics – Men's shot put", 'Summary')
('United States at the 2020 Summer Olympics', 'Medalists')
("Swimming at the 2020 Summer Olympics – Women's 200 metre backstroke", 'Records')
("Athletics at the 2020 Summer Olympics – Women's 100 metres hurdles", 'Records')
("Athletics at the 2020 Summer Olympics – Women's triple jump", 'Records')
("Gymnastics at the 2020 Summer Olympics – Men's pommel horse", 'Schedule')

Q: What was unusual about the men’s shotput competition?
A: The same three competitors received the same medals in back-to-back editions of an the same individual event.


In [64]:
query = "In the 2020 Summer Olympics, how many silver medals did Italy win?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 6 document sections:
('Italy at the 2020 Summer Olympics', 'Summary')
('List of 2020 Summer Olympics medal winners', 'Summary')
('United States at the 2020 Summer Olympics', 'Medalists')
('Rome bid for the 2020 Summer Olympics', 'Summary')
('List of divers at the 2020 Summer Olympics', 'Summary')
('Switzerland at the 2020 Winter Youth Olympics', 'Summary')

Q: In the 2020 Summer Olympics, how many silver medals did Italy win?
A: 10


Our Q&A model is less prone to hallucinating answers, and has a better sense of what it does or doesn't know. This works when the information isn't contained in the context; when the question is nonsensical; or when the question is theoretically answerable but beyond GPT-3's powers!

In [65]:
query = "What is the total number of medals won by France, multiplied by the number of Taekwondo medals given out to all countries?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 8 document sections:
('Taekwondo at the 2020 Summer Olympics – Qualification', 'Qualification summary')
('Taekwondo at the 2020 Summer Olympics', 'Summary')
('Brazil at the 2020 Summer Olympics', 'Competitors')
('United States at the 2020 Summer Olympics', 'Medalists')
('France at the 2020 Summer Olympics', 'Taekwondo')
('France at the 2020 Summer Olympics', 'Trampoline')
('Karate at the 2020 Summer Olympics', 'Qualification')
('South Korea at the 2020 Summer Olympics', 'Karate')

Q: What is the total number of medals won by France, multiplied by the number of Taekwondo medals given out to all countries?
A: I don't know.


In [66]:
query = "What is the tallest mountain in the world?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 9 document sections:
('2020 Summer Olympics closing ceremony', 'Weather conditions')
("Athletics at the 2020 Summer Olympics – Men's 400 metres hurdles", 'New records')
("Athletics at the 2020 Summer Olympics – Women's 1500 metres", 'Records')
("Athletics at the 2020 Summer Olympics – Women's high jump", 'Records')
("Athletics at the 2020 Summer Olympics – Women's 100 metres hurdles", 'Records')
("2020 CONCACAF Women's Olympic Qualifying Championship", 'Goalscorers')
('Kyrgyzstan at the 2020 Summer Olympics', 'Athletics')
("Athletics at the 2020 Summer Olympics – Women's triple jump", 'Records')
('2020 Summer Olympics opening ceremony', 'Weather conditions')

Q: What is the tallest mountain in the world?
A: I don't know.


In [67]:
query = "Who won the grimblesplatch competition at the 2020 Summer Olympic games?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 10 document sections:
("Wrestling at the 2020 Summer Olympics – Women's freestyle 62 kg", 'Summary')
("Wrestling at the 2020 Summer Olympics – Women's freestyle 50 kg", 'Summary')
("Wrestling at the 2020 Summer Olympics – Women's freestyle 53 kg", 'Summary')
("Wrestling at the 2020 Summer Olympics – Women's freestyle 68 kg", 'Summary')
("Wrestling at the 2020 Summer Olympics – Women's freestyle 57 kg", 'Summary')
("Wrestling at the 2020 Summer Olympics – Women's freestyle 76 kg", 'Summary')
('United States at the 2020 Summer Olympics', 'Medalists')
("Wrestling at the 2020 Summer Olympics – Men's freestyle 65 kg", 'Summary')
("Wrestling at the 2020 Summer Olympics – Men's freestyle 97 kg", 'Summary')
("Wrestling at the 2020 Summer Olympics – Men's freestyle 57 kg", 'Summary')

Q: Who won the grimblesplatch competition at the 2020 Summer Olympic games?
A: I don't know.
